## **Configuring the account**

In [ ]:
import subprocess, os
# identifying into github
!git config --global user.name "Nuzz23"
!git config --global user.email "nunzio.licalzi9@gmail.com"
output = subprocess.check_output("git config --global --list", shell=True).decode('utf-8').split()

# check if correctly identified
assert len(output) >= 2, "Wrong lenght"
assert output[0].split('=')[-1] == 'Nuzz23', 'wrong user name'
assert output[1].split('=')[-1] == 'nunzio.licalzi9@gmail.com', 'wrong email'
del output

In [4]:
with open("TOKENS.txt", 'r', encoding='utf-8') as f:
  for line in f:
    os.environ[line.split('=')[0].strip()] = line.strip().split('=')[1].strip()

## **Cloning the repository**

In [5]:
# if the cloned repository already exists delete it
%cd /content
%rm -rf sample_data
if "MLDL_SemanticSegmentation" in subprocess.check_output("ls", shell=True).decode("utf-8").strip():
  !rm -rf MLDL_SemanticSegmentation

/content


In [6]:
# clone the repository via token
!git clone --quiet "{os.getenv("GITHUB_TOKEN")}"

# check if cloned correctly
assert "MLDL_SemanticSegmentation" in subprocess.check_output("ls", shell=True).decode("utf-8").strip(), "Not cloned correctly"

# **LIBRARIES**

In [ ]:
# installing the required libraries
%cd /content/MLDL_SemanticSegmentation
!pip install -q -r requirements.txt

## **CUSTOM IMPORTS**

In [9]:
from stats import countFLOPS, latency, evaluateLastEpoch
from train.trainAdversarial import init_model
from datasets.downloader import Downloader

# **DATASET**

## **Downloading the dataset**

In [10]:
if not Downloader().downloadCityScapes():
  raise FileNotFoundError("CityScapes dataset not found")

In [11]:
if not Downloader().downloadGTA5():
  raise FileNotFoundError("GTA5 dataset not found")

# **MODEL**

In [12]:
from datasets.dataAugmentation.horizontalFlip import HorizontalFlip
from datasets.dataAugmentation.saltAndPepper import SaltAndPepper
from datasets.dataAugmentation.gaussianBlur import GaussianBlur
from datasets.dataAugmentation.colorJitter import ColorJitter

In [ ]:
TRAIN_SIZE, VAL_SIZE = (1280, 720), (1024, 512)

model, discriminator = init_model('bisenet', batchSize=4, learning_rate=5e-4, trainSize=TRAIN_SIZE, valSize=VAL_SIZE, momentum=0.9,
                   augmentation=ColorJitter(p=0.5, brightness=0, contrast=0, saturation=0, hue=0.1),
                   enablePrint=True, pushWeights=True, enablePrintVal=True, restartTraining=True, runId='6t5ocjb6')

flops = countFLOPS(model, width=VAL_SIZE[0], height=VAL_SIZE[1])
latencyMean, latencyStd, fpsMean, fpsStd = latency(model, width=VAL_SIZE[0], height=VAL_SIZE[1])

evaluateLastEpoch(model)